# <h1><center id="c1"> 🤹‍♀️ `Prompt` `Engineering` - выжми из модели максимум </center></h1>

<img src='../images/rob.png' align="right" width="528" height="528" >

## Оглавление ноутбука:

 * <a href="#c1"> 🚀Введение  </a>
   * <a href="#c2"> ❄️ Структура промпта </a> 
* <a href="#look1"> 🌡 Generation Temperature - настраиваем "творческость" модели!  </a>
* <a href="#check1"> 👯‍♀️ Добавление нескольких контекстов  </a>
* <a href="#check2"> 🧐 Как посчитать расходуемые токены и деньги 💸? </a>
* <a href="#check3"> 🤹‍♀️ Техники Prompt Engineering и кто такой Prompt Engineer 👨‍🔧?</a>
* <a href="#6">🧸 Выводы и заключения</a>

<div class="alert alert-info">

В этом ноутбуке попытаемся разобраться, что такое `Prompt Engineering` (дизайн промпта) и почему так **важно** в нём разобраться в первую очередь, начиная изучение сферы LLM моделей.<br> *Начнём с определений:*




***Промпт (prompt)*** - это запрос, который мы подаем большой языковой модели, чтобы получить нужную информацию или выполнить определенную задачу. Он определяет, как именно LLM будет интерпретировать нашу просьбу и формировать ответ. По сути, хорошо сформулированный промпт выступает своего рода ключом, открывающим доступ к возможностям языковой модели.

<img src='../images/prompt_def.webp' align="right" width="628" height="628" >

***Prompt Engineering (дизайн промпта)*** — это практика разработки и уточнения промптов (вопросов или инструкций) для получения конкретных ответов от моделей ИИ (не только языковых, но и от других, например, генерации картинок). Думайте об этом, как об интерфейсе между намерениями человека и результатами работы машины. В обширной сфере искусственного интеллекта, где модели обучаются на огромных наборах данных, правильный промпт может стать решающим фактором между пониманием моделью вашего запроса или его неверной интерпретацией. Например, если вы когда-либо взаимодействовали с голосовыми помощниками, такими как `Siri` или `Алиса`, вы занимались базовой формой `Prompt Engineering'а`.

<div class="alert alert-success">
    
На примере ниже, вы можете увидеть, как незначительные изменения в тексте промпта, могут значительно изменить результат генерации. <br>
Запросы к генеративной нейросети [Midjourney](https://docs.midjourney.com/docs/multi-prompts).

<img src='../images/cheesecake.png' align="center">

## <center id="c2"> ❄️ Упрощенная структура промпта 
    
<img src='../images/prompt_engineering_visual.png' align="center">

<div class="alert alert-info">

Компоненты, из которых может быть составлен промпт:

* **Инструкции (Instructions)** - Этот компонент обеспечивает модели ясные указания о том, что ожидается от нее в ответе. Инструкция определяет, как модель должна интерпретировать введенные данные и какие параметры ответа следует учесть. Например, инструкция может содержать информацию о стиле ответа, длине текста или других ограничениях.
* **Внешний контекст (External information or context)** - Этот компонент предоставляет модели дополнительный контекст или информацию, которую она может использовать при формировании ответа. Это может включать в себя факты, данные, или ссылки на внешние источники информации, которые могут быть полезными для модели. Например, если вы общались с `ChatGPT`, то вся история вашего диалога сохранялась и использовалась в качестве контекста при каждом новом сообщении, чтобы модель понимала о чём вы с ней разговаривали и не перескакивала с темы на тему.
* **Ввод пользователя или запрос (User input or query)** - Это входные данные, которые пользователь предоставляет модели. Это может быть вопрос, просьба или какой-либо запрос, который пользователь хочет, чтобы модель обработала.
* **Выходной индикатор (Output indicator)** -  Этот компонент указывает, как модель должна сформировать свой ответ. Это может быть, например, просьба о предоставлении ответа в определенной форме или формате, такой как краткое резюме, расширенное объяснение, код или что-либо еще.

Не всегда нужно указывать все компоненты для хорошего промпта, но помните, что корректное использование и балансирование этих компонентов помогает точно настраивать модель и получать желаемые результаты.

In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install langchain openai langchain-openai tiktoken -q

# Для работы в колабе загрузите наш скрипт для использования ChatGPT на сервере курса!
#!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [2]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите API ключ')


# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите API ключ ········


Напишем промпт

In [3]:
prompt = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: На каких онлайн платформах можно размещать курсы?

Answer: """

<div class="alert alert-info">

В этом примере мы имеем:
* Инструкцию
* Контекст
* Вопрос (запрос пользователя)
* Выходной индикатор ("Answer: ")

In [5]:
print(llm.invoke(prompt))

content='Coursera, Udemi, Stepik.'


<div class="alert alert-info">
    
В качестве альтернативы, если у нас нет нужной информации в `контексте`, модель должна ответить `"Я не знаю"`, давайте попробуем.

In [22]:
prompt = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В Интернете много сайтов.

Question: На каких онлайн платформах можно размещать курсы?

Answer: """


In [24]:
print(llm.invoke(prompt))

content='Я не знаю'


<div class="alert alert-info">

Отлично, модель понимает наши инструкции. В большинстве реальных случаев мы не будем предоставлять внешнюю информацию/контекст модели вручную. Это будет автоматический процесс, использующий что-то вроде долговременной памяти для извлечения соответствующей информации из внешнего источника (об этом подробнее в 4-м модуле курса).


## <center id="check1"> 👯‍♀️ Добавление нескольких контекстов </center>

<div class="alert alert-info">

В некоторых юзкейсах, таких как вопрос-ответ, мы можем использовать внешний источник информации, чтобы повысить надежность или фактологичность ответов модели. Мы называем эту информацию `source knowledge`, то есть любыми знаниями, введенными в модель через промпт.

Создадим список «фиктивной» внешней информации, воспользуемся строчками из документации `OpenAI`.

In [25]:
contexts = [
    (
        "Large Language Models (LLMs) are the latest models used in NLP. " +
        "Their superior performance over smaller models has made them incredibly " +
        "useful for developers building NLP enabled applications. These models " +
        "can be accessed via Hugging Face's `transformers` library, via OpenAI " +
        "using the `openai` library, and via Cohere using the `cohere` library."
    ),
    (
        "To use OpenAI's GPT-3 model for completion (generation) tasks, you " +
        "first need to get an API key from " +
        "'https://beta.openai.com/account/api-keys'."
    ),
    (
        "OpenAI's API is accessible via Python using the `openai` library. " +
        "After installing the library with pip you can use it as follows: \n" +
        "```import openai\nopenai.api_key = 'YOUR_API_KEY'\nprompt = \n" +
        "'<YOUR PROMPT>'\nres = openai.Completion.create(engine='text-davinci" +
        "-003', prompt=prompt, max_tokens=100)\nprint(res)"
    )
]

<div class="alert alert-info">

Нам нужно ввести внешнюю информацию в наш промпт между первоначальными инструкциями и пользовательским вводом. Для моделей `OpenAI` рекомендуется отделять контексты от остальной части подсказки с помощью `###` или `"""`, а каждый независимый контекст можно отделить несколькими символами новой строки `\n` и `##`, например:

In [29]:
# Воспользуемся методом join для объединения контекстов в 1 строку с разделителем
context_str = '\n\n##\n\n'.join(contexts)

# Напишем промпт
prompt = f"""Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

###

Contexts:
{context_str}

###

Question: Дай мне 2 примера как использовать GPT-3 на Python

Answer: """

In [30]:
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)
print(llm.invoke(prompt).content)

1. Установите библиотеку `openai` с помощью pip и получите API ключ от OpenAI. Затем используйте функцию `openai.Completion.create()` для создания запроса к GPT-3 модели и получения ответа.
2. Создайте переменную `prompt`, в которой будет содержаться текст, по которому GPT-3 будет генерировать продолжение. Передайте эту переменную в функцию `openai.Completion.create()` в качестве аргумента `prompt`.


<div class="alert alert-info">

Неплохо, но действительно ли эти контексты помогают? Возможно, модель способна ответить на эти вопросы без дополнительной информации (`source knowledge`), поскольку может полагаться исключительно на информацию, хранящуюся во внутренних параметрах модели (`parametric knowledge`). Давайте еще раз спросим без внешней информации.

In [23]:
prompt = """
Ответь на вопрос.
Если на вопрос нельзя ответить, ответь 'Я не знаю'.

Question: Дай мне 2 примера как использовать GPT-3 на Python

Answer: """

llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)
print(llm.invoke(prompt).content)

1) Один из примеров использования GPT-3 на Python может быть создание чат-бота, который сможет поддерживать разговор с пользователем и отвечать на его вопросы. GPT-3 может быть обучен на большом количестве различных текстовых данных, чтобы обеспечить более точные и информативные ответы.
2) Другой пример использования GPT-3 на Python - это создание системы автоматического резюме. GPT-3 может быть обучен на большом количестве данных резюме и может генерировать краткую и информативную сводку профессионального опыта и навыков на основе предоставленных данных.


<div class="alert alert-info">

Это не совсем то, что мы просили, и определенно не очень конкретное. Таким образом, очевидно, что добавление некоторых исходных знаний к нашим подсказкам может привести к гораздо лучшим результатам. В следующих уроках мы научимся, как проще добалять контекст к промптам и даже использовать внешние базы знаний.

## <center id="look1"> 🌡 Generation Temperature - настраиваем "творческость" модели! </center>

Параметр `temperature`, используемый в моделях генерации, говорит нам, насколько «случайной» может быть модель. Он представляет собой вероятность того, что модель выберет слово, которое не является первым выбором модели. Это работает, потому что модель фактически назначает прогноз вероятности для всех токенов в своем словаре на каждом «шаге» модели (каждом новом слове или подслове).

<div class="alert alert-info">
Вкратце, увеличение температуры может привести к большей случайности, что способствует более разнообразным или творческим результатам. Вы фактически увеличиваете веса других возможных токенов. В плане применения, для задач, связанных с ответами на вопросы на основе фактов, рекомендуется использовать более низкое значение температуры, чтобы стимулировать более точные и краткие ответы. Для генерации стихов или других творческих задач может быть полезно увеличить значение температуры.

Давай попробуем:

In [47]:
prompt = """Ниже представлен диалог с весёлым чатботом.
Ответы чатбота остроумные и забавные.

Chatbot: Привет! Я чатбот.
User: Привет, как дела?
Chatbot: """

llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)
print(llm.invoke(prompt).content)

У меня всегда хорошо, я же не настоящий человек! А у тебя как?


In [48]:
llm = ChatOpenAI(temperature=1.0, course_api_key=course_api_key)
print(llm.invoke(prompt).content)

Прекрасно! Я всегда в отличном настроении, потому что мне не нужно спать, есть или ходить в туалет. Как могу помочь тебе сегодня?


<div class="alert alert-info">

Второй ответ более креативен и демонстрирует разницу, которую мы можем ожидать между низкотемпературными и высокотемпературными генерациями.

# <center id="check2"> 🧐 Как посчитать расходуемые токены и деньги 💸?

<div class="alert alert-info">

Учитывая, что нам может потребоваться ввести внешнюю информацию в наши промпты, они могут стать довольно большими. Максимальное контекстное окно LLM относится к токенам как в промпте, так и в тексте ответа. Для `text-davinci-003` это 4097 токенов. Однако измерение общего количества входных токенов является более сложным.
Поскольку токены не сопоставляются непосредственно со словами, мы можем измерить количество токенов в тексте только путем фактической токенизации текста. Модели GPT используют токенизатор `TikToken` от `OpenAI`. Мы можем установить библиотеку через `Pip`:

Используя токенайзер, можем посчитать количество токенов в предыдущем промпте:

In [16]:
import tiktoken

prompt = f"""Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

###

Contexts:
{context_str}

###

Question: Дай мне 2 примера как использовать GPT-3 на Python

Answer: """

encoder_name = 'p50k_base'
tokenizer = tiktoken.get_encoding(encoder_name)

len(tokenizer.encode(prompt))

440

<div class="alert alert-info">

Это мы получили количество токенов только в промпте, количество токенов для ответа модели регулируется праметром `max_tokens` (по умолчанию 256), указывается максимально возможная длина, т.е. не обязательно, что модель всегда будет расходовать весь доступный лимит токенов, но точно не будет превышать.

In [17]:
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key, max_tokens=512)
print(llm.invoke(prompt).content)

1. Установите библиотеку `openai` с помощью pip: `pip install openai`
   Затем импортируйте ее и укажите ваш API-ключ:
   ```python
   import openai
   openai.api_key = 'YOUR_API_KEY'
   ```

2. Создайте запрос к модели GPT-3, используя метод `Completion.create`:
   ```python
   prompt = '<YOUR PROMPT>'
   res = openai.Completion.create(engine='text-davinci-003', prompt=prompt, max_tokens=100)
   print(res)
   ```
   Здесь `prompt` - это текст, который вы хотите передать в модель, а `max_tokens` - это максимальное количество токенов в ответе модели.


In [18]:
len(tokenizer.encode(llm.invoke(prompt)))

1275

<div class="alert alert-info">

Получаем 440 + 504 ~ 1000 токенов на 1 вопрос-ответ, при стоимости 1000 токенов `0.002$`. Т.е. 10 пользователей за 100 запросов сожгут уже `2$`. Причем окно контекста используется только на 1\3. <br>
Начинаем продумывать экономику своего сервиса 🤑

# <center id="check3"> 🤹‍♀️ Техники prompt engineering и кто такой Prompt Engineer 👨‍🔧? 

<div class="alert alert-success">
    
#### **Базовые техники** (советы, которые обычный пользователь может использовать, чтобы улучшить свои промпты):
*  **Ролевая игра** Заставив модель действовать как конкретная сущность, например, историк или ученый, вы можете получить индивидуальные ответы. Например, фраза «Как диетолог, оцените следующий план диеты» может привести к ответу, основанному на науке о питании.
* **Итеративное уточнение** Начните с общей фразы запроса и постепенно уточняйте ее на основе ответов модели. Этот итеративный процесс помогает довести промпт до совершенства.
* **Цикл обратной связи** Используйте выходные данные модели для информирования и корректировки последующих промптов. Такое динамическое взаимодействие гарантирует, что ответы модели с течением времени будут более точно соответствовать ожиданиям пользователей.

<div class="alert alert-info">
    
#### **Продвинутые техники** (более сложные стратегии, требующие более глубокого понимания поведения модели. Будем рассматривать дальше в курсе):
*  **Семантический поиск** - этот метод предполагает предоставление модели релевантного фрагмента для использования при ответе. Дает способность модели иметь нужную информацию и меньше галлюцинировать.
*  **Few-shot prompting** Здесь модели дается несколько примеров (shots), которые помогут ей отреагировать. Предоставляя контекст или предыдущие экземпляры, модель может лучше понять и сгенерировать желаемый результат. Например, можно показать модели несколько примеров переведенных предложений, прежде чем попросить ее перевести новое.
* **Chain-of-Thought (Цепочка мыслей)**. Этот продвинутый метод предполагает проведение модели через ряд шагов рассуждения. Разбивая сложную задачу на промежуточные этапы или «цепочки рассуждений», модель может добиться лучшего понимания языка и более точных результатов. Это похоже на пошаговое руководство для решения сложной математической задачи.
* **И другие**, но про них уже будем говорить по факту использования в продвинутых модулях

<div class="alert alert-custom">

Быстрый прогресс в области обработки естественного языка (NLP) и широкое распространение больших языковых моделей (LLM) создали новую нишу и большой спрос на экспертов, которые могут создавать эффективные промпты. Эти люди, известные как **промпт инженеры**, являются не просто техническими специалистами, но и творцами, которые понимают нюансы языка, контекста и поведения ИИ.

<div class="alert alert-custom">


Как сообщается в журнале Time, компании, от технологических гигантов до стартапов, осознают ценность специализированных должностей в области Prompt Engeneering'а. Поскольку решения на основе искусственного интеллекта все больше интегрируются в продукты и услуги, опыт **промпт инженера** гарантирует, что эти решения будут эффективными, удобными для пользователя и релевантными контексту.Такие сайты вакансий, как Indeed и LinkedIn, уже публикуют только в США тысячи вакансий на роль **промпт инженера** , с зарплатами от 50 000 до более 150 000 долларов в год. <br>
Российский рынок тоже начинает реагировать на новый мировой тренд, можете почитать [обзор](https://vc.ru/chatgpt/823927-kto-takie-prompt-inzhenery-i-mozhno-li-osvoit-etu-specialnost-ne-znaya-yazykov-programmirovaniya), там их называют **"промптёры"**.

# <center id="6"> 🧸 Выводы и заключения

Подводя итог, можно выделить несколько советов для базового промптинга, следуя которым можно сразу значительно поднять релевантность получаемых результатов. 

## 1. Создайте адекватный промпт.
* **Ясность является ключевым моментом.** Убедитесь, что промпт ясен и недвусмысленнен. Избегайте жаргона, если это не необходимо для контекста.
* **Попробуйте сыграть в ролевую игру.** Как обсуждалось ранее, если модель возьмет на себя конкретную роль, это может привести к более адаптированным ответам.
* **Используйте ограничения.** Установка границ или ограничений может помочь направить модель к желаемому результату. Например, «Опишите Эйфелеву башню в трех предложениях» обеспечивает четкое ограничение длины.
* **Инструкция**. Вы можете разрабатывать эффективные промпты для различных простых задач, используя команды для указания модели, что вы хотите достичь, такие как "Write", "Classify", "Summarize", "Translate", "Order", и т.д. Еще одна рекомендация состоит в использовании некоторого ясного разделителя, например "###", для отделения инструкции и контекста.
* **Избегайте наводящих вопросов.** Наводящие вопросы могут исказить результаты модели. Очень важно сохранять нейтралитет, чтобы получить объективный ответ.
* **Избегайте неточностей**. Учитывая вышеуказанные рекомендации о детализации и улучшении формата, легко попасть в ловушку желания быть слишком умным в промптах и, возможно, создавать неточные описания. Часто лучше быть конкретным и прямым. Аналогия здесь очень похожа на эффективную коммуникацию - чем прямее, тем эффективнее передается сообщение. Возможно, с помощью таких промптов вы все равно получите неплохие ответы, но лучший промпт будет очень конкретным, кратким и по существу.
* **Делать или не делать?** - Избегайте формулировки того, что не нужно делать, а вместо этого указывайте, что нужно делать. Это способствует большей специфичности и фокусу на деталях, которые приводят к хорошим результатам модели.

## 2. Повторяем и оцениваем

Процесс уточнения подсказок является итеративным. Вот типичный рабочий процесс:

* **Составьте начальный промпт.** В зависимости от поставленной задачи и желаемого результата.
* **Проверьте промпт.** Используйте модель ИИ для генерации ответа.
* **Оцените результат.** Проверьте, соответствует ли ответ намерению и критериям.
* **Уточните промпт.** Внесите необходимые корректировки на основе оценки.
* **Повторить.** Продолжайте этот процесс до тех пор, пока не будет достигнуто желаемое качество вывода.

## 3. Калибровка и точная настройка модели.

Помимо доработки самого промпта, существует также возможность калибровки или точной настройки модели ИИ. Это предполагает настройку параметров модели для лучшего соответствия конкретным задачам, наборам данных или экономике проекта (например, `temperature`, `top_n`, `max_tokens`). Хотя это более продвинутый метод, он может значительно улучшить производительность модели для специализированных приложений.
